# Metrics Pipeline Template

Track notebook analytics updates with the digest-agent bridge.

1. Load sanitized pipeline metrics from `.workspace/metrics/`.
2. Optionally call `/knowledge/*` endpoints to enrich results.
3. Persist the refreshed metrics under `storage/analytics/pipelines/`.


In [ ]:
import json
from pprint import pprint
from notebooks.lib.research_bridge import ResearchBridge, GatewayRequestError

bridge = ResearchBridge()
sample_metrics_path = bridge.sample_dataset_path("pipeline_metrics_sample.json")
sample_metrics = json.loads(sample_metrics_path.read_text())
pprint(sample_metrics)

try:
    synthesis = bridge.synthesize_knowledge(
        topic="digest-agent run recap",
        nodes=sample_metrics.get("latest_runs", []),
        options={"focus": "risk"},
    )
except GatewayRequestError as exc:
    print(f"Gateway offline or unreachable: {exc}")
    synthesis = {"message": "offline demonstration"}

refined_metrics = {
    **sample_metrics,
    "knowledge_summary": synthesis.get("message", "no gateway response"),
    "template": "metrics_pipeline",
}

artifact_path = bridge.push_metrics(
    pipeline_name=sample_metrics.get("pipeline", "digest-agent-metrics"),
    metrics=refined_metrics,
)
print(f"Metrics stored at: {artifact_path}")
